#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2019


# Homework 3:   Collaborative Filtering + Matrix Factorization

### 100 points [6% of your final grade]

### Due: Wednesday, March 27, 2019 by 11:59pm

*Goals of this homework:* Understand how collaborative filtering and matrix factorization works. Explore different methods for real-world recommendation senarios.

*Submission instructions (eCampus):* To submit your homework, rename this notebook as `UIN_hw3.ipynb`. For example, my homework submission would be something like `555001234_hw3.ipynb`. Submit this notebook via eCampus (look for the homework 3 assignment there). Your notebook should be completely self-contained, with the results visible in the notebook. We should not have to run any code from the command line, nor should we have to run your code within the notebook (though we reserve the right to do so). So please run all the cells for us, and then submit.

*Late submission policy:* For this homework, you may use as many late days as you like (up to the 5 total allotted to you).

*Collaboration policy:* You are expected to complete each homework independently. Your solution should be written by you without the direct aid or help of anyone else. However, we believe that collaboration and team work are important for facilitating learning, so we encourage you to discuss problems and general problem approaches (but not actual solutions) with your classmates. You may post on Piazza, search StackOverflow, etc. But if you do get help in this way, you must inform us by **filling out the Collaboration Declarations at the bottom of this notebook**. 

*Example: I found helpful code on stackoverflow at https://stackoverflow.com/questions/11764539/writing-fizzbuzz that helped me solve Problem 2.*

The basic rule is that no student should explicitly share a solution with another student (and thereby circumvent the basic learning process), but it is okay to share general approaches, directions, and so on. If you feel like you have an issue that needs clarification, feel free to contact either me or the TA.

# Part 1. Collaborative Filtering (50 points)

In this part, you will implement collaborative filtering on the [MovieLens Latest Datasets](http://files.grouplens.org/datasets/movielens/ml-latest-small-README.html). After removing users who left less than 20 ratings and movies with less than 20 ratings, the provided dataset has only ~1,200 items and ~500 users. You can also check the title and genres of each movie in *movies_info.csv*.

As background, read [Collaborative Filtering Recommender Systems](http://files.grouplens.org/papers/FnT%20CF%20Recsys%20Survey.pdf) and refer to the course slides `week06rec.pdf` for collaborative filtering.

## 1.1 Load the Data

Please download the dataset from Piazza. There are about 65,000 ratings in total. We split the rating data into two set. You will train with 70% of the data (in *train_movie.csv*) and test on the remaining 30% of data (in *test_movie.csv*). Each of train and test files has lines having this format: UserID, MovieID, Rating. 

First you will need to load the data and store it with any structure you like. Please report the numbers of unique users and movies in the dataset. 

In [103]:
import csv
import numpy as np
import math
from sklearn.decomposition import NMF
from copy import copy

In [ ]:
# movies and users in each of train and test sets.
# Your Code Here...
# use a user dictionary, key => user  value => another dictionary
# dict.get(i).get(j) =. user i's rating on movie j
def load_data(fileName):
    user_dict = {}
    movie_dict = {}
    with open(fileName) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter = ',')
        for counter, row in enumerate(csv_reader):
            if counter > 0: 
                user_temp = {int(row[1]): int(row[2])}
                movie_temp = {int(row[0]): int(row[2])}
                if int(row[0]) not in user_dict:
                    user_dict[int(row[0])] = {}
                if int(row[1]) not in movie_dict:
                    movie_dict[int(row[1])] = {}
                user_dict.get(int(row[0]))[int(row[1])] = int(row[2])
                movie_dict.get(int(row[1]))[int(row[0])] = int(row[2])
    return user_dict, movie_dict, counter
            

In [ ]:
def build_matrix(user_dict, movie_dict):
    # build the matrix, row represent users and col represent movies
    # need to notice that matrix[i][j] = 0 doesn't necessarily mean user i didn't rate movie j, maybe user i gives
    # movies j a 0
    res = np.zeros((len(user_dict), len(movie_dict)))
    for user, movies in user_dict.items():
        for movie in movies:
            res[user][movie] = user_dict.get(user).get(movie)
    return res



In [ ]:
train_user_dict, train_movie_dict, train_counter = load_data("train_movie.csv")
test_user_dict, test_movie_dict, test_counter = load_data("test_movie.csv")

In [100]:
train_matrix = build_matrix(train_user_dict, train_movie_dict)
test_matrix = build_matrix(test_user_dict, test_movie_dict)

In [ ]:
USER_NUM = len(train_user_dict)
MOVIE_NUM = len(train_movie_dict)

In [177]:
print(len(train_user_dict), len(train_movie_dict))
print(len(test_user_dict), len(test_movie_dict))

541 1211
541 1211


## 1.2 Implement User-based Collaborative Filtering

In this part, you will implement the basic User–User Collaborative Filtering algorithm introduced in the class. Given the ratings by each user, you are going to try different methods in calculating the similarities between users. You will use equation `(c)` in `week06rec.pdf` (Page 40) to aggregate ratings. Set k = 0.05. Just consider all users as neighbors. That is, while predicting how user $u$ will rate item $i$, $\widehat{C}$ includes all users who have ratings on i in the training set.

*For this memory-based algorithm, you can only rely on the ratings in training set to predict for the testing set. That is, you assume that you don't know any ratings information in the test set except that when you evalaute your model.*

### Cosine Similarity

First, you will try to calculate the similarity between users with cosine similary following the equation on page 16 of [Collaborative Filtering Recommender Systems](http://files.grouplens.org/papers/FnT%20CF%20Recsys%20Survey.pdf). And then you need to predict the rating for each (user, movie) tuple in the test set. *Note: you don't need to subtract user mean baseline from the ratings prior to computing the similarity.*

In [106]:
# Your Code Here...
# Predict for test set

def cos_similar(matrix, id_one, id_two):
    # given two users, calculate the cosine similarity
    a = np.dot(matrix[id_one], matrix[id_two])
    b = sum(np.power(matrix[id_one], 2))
    c = sum(np.power(matrix[id_two], 2))
    return a / (math.sqrt(b) * math.sqrt(c))

def predict(user, movie, matrix, train_user_dict, mean_dict):
    # aggregate the result using cosine similarity
    aggregate = 0
    for i in range(USER_NUM):
        if (i is not user) and contains(i, movie, train_user_dict):
            aggregate += (cos_similar(matrix, i, user) * (matrix[i][movie] - mean_dict.get(i)))
    return aggregate * 0.05 
                          
def average(user, user_dict):   
    # given a user, calculate the average rating he has on all movies
    total = 0
    movies = user_dict.get(user)
    for movie in movies:
        total += movies.get(movie)
    return total / len(movies)
    

def mean_map(user_dict):
    # build a dict that map each user with his average score
    res = {}
    total = 0
    for user, movies in user_dict.items():
        total = 0
        for movie in movies:
            total += movies.get(movie)
        res[user] = total / len(movies)
    return res;


def contains(user, movie, user_dict):
    # given a user and a movie, return true if this user rated this movie before
    if movie in user_dict.get(user):
        return True;
    return False


In [24]:
# build the mean dict
mean_dict = mean_map(train_user_dict)

In [25]:
# results for cosine
result = {}
for user, movies in test_user_dict.items():
    for movie in movies:
        temp = mean_dict.get(user) + predict(user, movie, train_matrix, train_user_dict, mean_dict)
        result[(user, movie)] = temp

### Evaluation

You should evaluate your predictions using Mean Absolute Error and Root Mean Squared Error. 

In [138]:
def evaluation(result, test_user_dict, test_counter):
    # given a result, evaluate the result
    MAE = 0
    RMSE = 0
    for user, movies in test_user_dict.items():
        for movie in movies:
            MAE += abs(movies.get(movie) - result.get((user, movie)))
            RMSE += pow((movies.get(movie) - result.get((user, movie))), 2)
    MAE /= test_counter
    RMSE = math.sqrt(RMSE / test_counter)
    return MAE, RMSE

In [178]:
# Your Code Here...
# Report Mean Absolute Error and Root Mean Squared Error for test set
cos_MAE, cos_RMSE = evaluation(result, test_user_dict, test_counter) 
print("User-based Collaborative Filtering with Cosine Similarity: ")
print("MAE: ", cos_MAE, "  RMSE: ", cos_RMSE)  # 0.6903937198616398 0.8991489107420851


User-based Collaborative Filtering with Cosine Similarity: 
MAE:  0.6903937198616398   RMSE:  0.8991489107420851


### Pearson correlation

Then, you will try to calculate the similarity between users with pearson correlation following `week06rec.pdf` (Page 37). And then you need to predict the rating for each (user, movie) tuple in the test set.

In [180]:
# Your Code Here...
# Predict for test set


def pearson_correlation(user_dict, id_one, id_two, mean_dict):
    # given two users, calculate their pearson correlation
    a = 0
    b = 0
    c = 0
    for movie in user_dict.get(id_one):
        if movie in user_dict.get(id_two):
            a += (user_dict.get(id_one).get(movie) - mean_dict.get(id_one)) * (user_dict.get(id_two).get(movie) - mean_dict.get(id_two))
            b += pow(user_dict.get(id_one).get(movie) - mean_dict.get(id_one), 2)
            c += pow(user_dict.get(id_two).get(movie) - mean_dict.get(id_two), 2)
    if b * c == 0:
        return 0
    return a / math.sqrt(b * c)


def pearson_predict(user, movie, train_user_dict, mean_dict):
    # aggregate the result
    aggregate = 0
    for i in range(len(train_user_dict)):
        if (i is not user) and contains(i, movie, train_user_dict):
            aggregate += (pearson_correlation(train_user_dict, user, i, mean_dict) * (train_user_dict.get(i).get(movie) - mean_dict.get(i)))
    return aggregate * 0.05 
    


In [181]:
pearson_result = {}
for user, movies in test_user_dict.items():
    for movie in movies:
        temp = mean_dict.get(user) + pearson_predict(user, movie, train_user_dict, mean_dict)
        pearson_result[(user, movie)] = temp

### Evaluation

You should evaluate your predictions using Mean Absolute Error and Root Mean Squared Error. 

In [182]:
# Your Code Here...
# Report Mean Absolute Error and Root Mean Squared Error for test

pearson_MAE, pearson_RMSE = evaluation(pearson_result, test_user_dict, test_counter)   
print("User-based Collaborative Filtering with Pearson Similarity: ")
print("MAE: ", pearson_MAE, "  RMSE: ", pearson_RMSE)  # 0.6917506759527966 0.8983386235087089

User-based Collaborative Filtering with Pearson Similarity: 
MAE:  0.6917506759527966   RMSE:  0.8983386235087089


### Pearson correlation (varying the threshold)

In [Collaborative Filtering Recommender Systems](http://files.grouplens.org/papers/FnT%20CF%20Recsys%20Survey.pdf), they observe that: 

> Pearson correlation suffers from computing high similarity
between users with few ratings in common. This can be alleviated by setting a threshold on the number of co-rated items
necessary for full agreement (correlation of 1) and scaling the
similarity when the number of co-rated items falls below this
threshold.

So now revise your Pearson to consider a threshold. Try several values and report for one that you think is appropriate.

In [196]:
# Your Code Here...
# Predict for test set
def pearson_threshold(user_dict, id_one, id_two, mean_dict):
    a = 0
    b = 0
    c = 0
    common = 0
    for movie in user_dict.get(id_one):
        if movie in user_dict.get(id_two):
            common += 1
            a += (user_dict.get(id_one).get(movie) - mean_dict.get(id_one)) * (user_dict.get(id_two).get(movie) - mean_dict.get(id_two))
            b += pow(user_dict.get(id_one).get(movie) - mean_dict.get(id_one), 2)
            c += pow(user_dict.get(id_two).get(movie) - mean_dict.get(id_two), 2)
    if b * c == 0:
        return 0;
    elif common > 25:
        return a / math.sqrt(b * c)
    return (common / 25) * a / math.sqrt(b * c)

def pearson_threshold_predict(user, movie, train_user_dict, mean_dict):
    aggregate = 0
    for i in range(len(train_user_dict)):
        if (i is not user) and contains(i, movie, train_user_dict):
            aggregate += (pearson_threshold(train_user_dict, user, i, mean_dict) * (train_user_dict.get(i).get(movie) - mean_dict.get(i)))
    return aggregate * 0.05 


In [197]:
# Your Code Here...
# Report Mean Absolute Error and Root Mean Squared Error for test

threshold_result = {}
for user, movies in test_user_dict.items():
    for movie in movies:
        temp = mean_dict.get(user) + pearson_threshold_predict(user, movie, train_user_dict, mean_dict)
        threshold_result[(user, movie)] = temp

### Evaluation

You should evaluate your predictions using Mean Absolute Error and Root Mean Squared Error. 

In [198]:
threshold_MAE, threshold_RMSE = evaluation(threshold_result, test_user_dict, test_counter)  
print("User-based Collaborative Filtering with Pearson Similarity(Varying the Threshold): ")
print("MAE: ", threshold_MAE, "  RMSE: ", threshold_RMSE)  # 0.6917506759527966 0.8983386235087089

User-based Collaborative Filtering with Pearson Similarity(Varying the Threshold): 
MAE:  0.6883253384684946   RMSE:  0.8936574572009663


What do you observe? How different are your results for the original Pearson Correlation approach vs. the thresholded version vs. the Cosine Similarity approach? Why do you think this is? *provide a brief (1-2 paragraph) discussion based on these questions.*

#### Cosine: MAE:  0.6903937198616398   RMSE:  0.8991489107420851 
#### Pearson: MAE:  0.6917506759527966   RMSE:  0.8983386235087089 
#### Threshold: MAE:  0.6883253384684946   RMSE:  0.8936574572009663

Among these three results, the thresholded version is the best, the second one is the Cosine Similarity approach and Pearson Correlation is the worst among these three. 

Problem for Cosine Similarity is that it treats missing value as "Negative". This will greatly influence the result when you have a lot of missing values in your matrix. And the problem for Pearson Correlation is that it return a high similarity between two users with few ratings in common. And the Pearson Correlation with threshold solves this problem by setting a lower bound on number of common ratings between two users. 



# Part 2. MF (20 points)

In class, we introduced how matrix factorization works for recommendation. Now it is your term to implement it. There are different methods to obtain the latent factor matrices **P** and **Q**, like gradient descent, Alternating Least Squares (ALS), and so on. Pick one of them and implement your MF model. *You can refer to tutorials and resources online. Remember our **collaboration policy** and you need to inform us of the resources you refer to.* 

Please report MAE and RMSE of your MF model for the test set. *You are expected to get a lower MAE and RMSE compared with the results in Part 1.*

In [ ]:
a = {"k": "d"}
if a.get("k") is not None:
    print(a.get("k"))

In [188]:
# Your Code Here...
# Report Mean Absolute Error and Root Mean Squared Error for test
# gradient descent
def matrix_factorization(R, P, Q, K, steps, alpha, beta, user_dict):
    Q = Q.T
    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if user_dict.get(i).get(j) is not None:
                    eij = R[i][j] - np.dot(P[i,:], Q[:,j])
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = np.dot(P,Q)
    return P, Q.T
                        


Which method did you use to obtain **P** and **Q**? What are the advantages and disadvantages of the method you pick? *provide a brief (1-2 paragraph) discussion based on these questions.*

I used gradient descent to obtain the P and Q. The advantages of gradient descent is that it is easy to understand and implement. And it can provide a better result compared with user-based collaborative fitering.  
The downside of the gradient descent is that the convergence rate is slow if you set the learning rate slow. And it might never converge if you set the learning rate too big. So it requires manual parameter tuning. Even if you have a proper learning rate, it is still a long process if you have a large dataset. 

In [191]:
N = len(train_matrix)
M = len(train_matrix[0])
K = 2

P = np.random.rand(N,K)
Q = np.random.rand(M,K)
alpha = 0.00002
beta = 0.02
steps = 1000
nP, nQ = matrix_factorization(train_matrix, P, Q, K, steps, alpha, beta, train_user_dict)
nR = np.dot(nP, nQ.T)

In [186]:
# Evaluation
def MFEval(test_user_dict, result, test_counter):
    MAE = 0
    RMSE = 0
    for user, movies in test_user_dict.items():
        for movie in movies:
            MAE += abs(movies.get(movie) - result[user][movie])
            RMSE += pow((movies.get(movie) - result[user][movie]), 2)
    MAE /= test_counter
    RMSE = math.sqrt(RMSE / test_counter)
    return MAE, RMSE

In [199]:
MF_MAE, MF_RMSE = MFEval(test_user_dict, nR, test_counter)
print("MF using Gradient Descent")
print("MAE: ", MF_MAE, " RMSE: ",  MF_RMSE)

MF using Gradient Descent
MAE:  0.687973222295372  RMSE:  0.8921783968682473


# Part 3. Extension (30 points)

Given your results in the previous parts, can you do better? For this last part you should report on your best attempt at improving MAE and RMSE. Provide code, results, plus a brief discussion on your approach. Hints: You may consider using the title or genres information, trying other algorithms, designing a hybrid system, and so on. *As in the last homework, you can do anything you like to improve MAE and RMSE.*

You will get full marks for this part if you get better results than both of your CF and MF (of course we will also judge whether what you do here is reasonable or not). Additionally, you will get 5 points as bonus if your model performs the best among the whole class.

## Method
predict value = overall average + user bias with Jaccard Similarity in movie genre + item bias



In [73]:
# Your Code Here...
# Report Mean Absolute Error and Root Mean Squared Error for test

# map each movie with a set of genres
genre_map = {}
with open ('movies_info.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter = '\t')
    for counter, row in enumerate(csv_reader):
        if counter > 0:
            genres = row[2].split('|')
            genre_map[counter - 1] = genres


In [112]:
def overall_average(dict):
    # calculate the overall mean
    sum = 0
    count = 0
    for user, movies in dict.items():
        for movie in movies:
            sum += dict.get(user).get(movie)
            count += 1
    return sum / count

def get_bias(dic, item):
    sum = 0
    count = 0
    for i in dic.get(item):
        sum += dic.get(item).get(i)
        count += 1
    return sum / count

def jaccard(genre_map, i, j):
    return len(list(set(genre_map.get(i)).intersection(genre_map.get(j)))) / len(list(set(genre_map.get(i)).union(genre_map.get(j))))

In [93]:
mean = overall_average(train_user_dict)
print(mean)

3.4825096064661456


In [174]:
def cal_user_bias(train_user_dict, i, genre_map, j):
    # use jaccard similarity to calculate user bias
    norm = 0
    total = 0
    count = 0
    for movie in train_user_dict.get(i):
        similar = jaccard(genre_map, movie, j)
        norm += similar
        count += 1
    norm /= count
    if norm == 0:
        return 0
    for movie in train_user_dict.get(i):
        similar = jaccard(genre_map, movie, j)
        total += train_user_dict.get(i).get(movie) * (similar / norm)
    return total / count
    

In [175]:
better_result = {}
for user, movies in test_user_dict.items():
    for movie in movies:
        user_bias = cal_user_bias(train_user_dict, user, genre_map, movie) - mean
        movie_bias = get_bias(train_movie_dict, movie) - mean
        better_result[(user, movie)] = mean + user_bias + movie_bias
        
     

0 635 0.9261844186535382 -0.31924430034369644
0 587 0.8671663291391245 -0.7110810350375742
0 339 0.9704320595070608 0.003204679248140163
0 250 1.1793176776389864 -0.14917627313281212
0 411 0.875998850170165 0.017490393533854398
0 324 1.006173494493527 0.2049903935338544
0 589 0.8826082590686108 -0.42536674932328866
0 267 0.8936149339390789 0.594413470456931
0 382 0.9907672189133776 0.20714556594764755
0 473 0.9385496872175136 -0.26432778828432735
0 639 0.946102001818184 0.017490393533854398
0 396 0.946102001818184 -0.577747701704241
0 251 1.1771389306949671 -0.2902019141584531
0 569 0.881286733280453 0.5636442396877004
0 480 1.0037641217985187 0.3905062665497274
0 203 0.9143098649389629 -0.24942689969922816
0 629 0.946102001818184 0.2374903935338546
0 663 1.0141655853497116 0.5293951554386163
0 577 0.881286733280453 0.4341570602005209
0 621 1.0199890054161442 0.2097980858415469
0 258 0.9596089072174752 -0.05393817789471722
0 519 0.8887583043377298 -0.34457857198338715
0 614 0.959948239

14 922 0.014970397916454825 0.017490393533854398
14 619 0.27942695446874577 0.45951937904110096
14 1195 -0.07975619049102622 0.0487403935338544
14 1203 0.13511568429300214 0.2362403935338544
14 91 -0.1497146850680262 0.4430223084274716
14 99 0.24818845913520082 0.8616764400454824
14 1064 -0.0593182577695095 0.47749039353385436
14 80 0.023295167379867365 0.6841570602005214
14 0 -0.3202537605239182 0.37360550144752347
14 608 0.03206727691878841 0.27868442338460087
14 767 -0.03361108714509253 -0.7444143683709075
14 707 -0.02279687218693649 0.14906934090227564
14 717 -0.04590751678395 0.3258237268671875
14 811 -0.1358347861022855 -0.2602873842439233
14 890 -0.29083093516710257 0.2648588145864861
14 1202 0.023295167379867365 -0.020971144927683927
14 491 -0.3268096902884423 -0.22325034720688652
14 1151 -0.0028717084238509827 -0.4825096064661456
14 331 -0.14798888107754626 0.36285121827612254
14 945 -0.07975619049102622 -0.28685743255310214
14 1183 -0.025805423264011917 0.284157060200521
14 4

18 136 -0.5708387903796353 -0.597894221850761
18 0 -0.5821326586395106 0.37360550144752347
18 431 -0.6295941063005768 0.13513745235738384
18 705 -0.7105515275276382 0.0545274305708916
18 741 -0.7105515275276382 -1.1491762731328121
18 188 -0.4570030941651315 0.08891896496242602
18 466 -0.6402468379220103 -0.5450096064661456
18 396 -0.6007214322869117 -0.577747701704241
18 358 -0.7476796274350366 -0.30859656298788485
18 731 -0.5784012792738089 0.4460618221052828
18 427 -0.6007214322869117 -0.5859578823282146
18 289 -0.598845793797881 0.5583067200644662
18 409 -0.7370684447570426 -0.7456675012029876
18 447 -0.6702597880651444 -0.5658429397994791
18 143 -0.6297933086794258 0.05595193199539272
18 445 -0.7370684447570426 -1.0088253959398297
18 171 -0.5887299189923767 0.2503148210147703
18 572 -0.7370684447570426 -0.3916005155570548
18 80 -0.5708387903796353 0.6841570602005214
18 711 -0.7341382709612603 -0.2825096064661454
18 30 -0.7370684447570426 -1.0325096064661454
18 154 -0.73706844475704

22 301 0.029819160657144295 0.7079665840100451
22 217 0.029819160657144295 0.7026755787190395
22 130 0.029819160657144295 0.017490393533854398
22 332 0.01074665557624499 0.5174903935338544
23 869 -0.02835507740524834 0.21749039353385458
23 499 -0.00824720847493321 0.06294493898839981
23 918 -0.005711382147632005 0.1760269788997082
23 25 0.07845650183147734 0.6548949736865262
23 180 0.058297181482370064 0.5748674427141824
23 558 -0.07644440555636534 -0.6295684299955573
23 1145 -0.010942393610610779 0.05915706020052092
23 803 -0.07644440555636534 -0.30603901823085167
23 99 0.16204666068847384 0.8616764400454824
23 1021 0.02143772999266913 -0.07074490058379279
23 1023 0.07315590565656649 0.017490393533854398
23 1013 0.006266041178987436 -0.7325096064661456
23 828 -0.02487601867988598 0.40121132376641233
23 964 0.0013174279226855212 0.3091570602005209
23 305 -0.05607818169916756 0.19021766626112724
23 974 -0.008549395394727988 -0.2411302961213182
23 1113 0.05357919957017421 0.4237403935338

31 361 0.4616964021175458 0.3595956566917491
31 497 0.33113868749710873 -0.03806516202170096
31 92 0.4249890825742333 0.512705704538639
31 437 0.4695661400742783 -0.4325096064661458
31 608 0.4662458840170167 0.27868442338460087
31 284 0.48426090236044006 0.24825962430308524
31 605 0.33113868749710873 -0.8491762731328123
31 7 0.40661895263653136 0.25218427108487473
31 0 0.41028771346684945 0.37360550144752347
31 84 0.4119241747238722 0.24825962430308524
31 387 0.4119241747238722 0.2821962758867955
31 139 0.3155359961397175 0.3896708446616741
31 480 0.5313792824227463 0.3905062665497274
31 247 0.4361556698633624 -0.36486254764261616
31 113 0.41981203931099786 0.530823726867188
31 523 0.49198361410037705 0.02692435579800545
31 492 0.9230105633852337 -0.09541283227259711
31 150 0.42856911073502246 -0.07074490058379279
31 377 0.4616964021175458 -0.06432778828432761
32 172 -0.15292006184699058 0.34755575301097874
32 811 -0.47191306015531476 -0.2602873842439233
32 558 -0.8314170913568937 -0.6

...

528 409 0.7206433674745729 -0.7456675012029876
528 489 0.6863242520228874 -0.017393327396378222
528 299 0.2748716783045526 0.6130786288279721
528 139 0.3317027223009794 0.3896708446616741
528 66 0.7206433674745729 0.2187890948325557
528 501 0.5919493300395202 -0.3991762731328121
528 132 0.2885571984714641 -0.4825096064661456
528 389 0.7206433674745729 -0.5641422595273702
528 610 0.7206433674745729 -0.4055865295430685
528 598 0.6512483553172941 -0.3942743123484984
528 295 0.7860306391000234 0.7527845111809133
528 202 0.759845892375957 0.3587602348036958
528 166 0.4800503935338565 0.0799903935338544
528 75 1.5174903935338544 0.8508237268671874
528 318 0.9522497518226296 0.3674903935338545
528 221 0.6518028159076326 0.4852323290177254
528 432 0.6512483553172941 -0.39679532075185975
528 451 0.8397334844917324 0.6227535514285916
528 265 0.5390650643099817 0.5174903935338544
528 424 0.8024177846204856 0.4374903935338543
528 417 0.7516183734585717 0.11749039353385449
528 480 0.600315041737924

529 506 -0.7150725214311251 0.5651094411529018
529 418 -0.6183417384539736 0.4374903935338543
529 636 -0.6857640741863191 -1.3713984953550344
529 837 -0.738957887622373 -0.6325096064661455
529 445 -1.0232049330199793 -1.0088253959398297
529 517 -0.8254997971276126 0.5174903935338544
529 682 -0.6541929803897664 0.4174903935338543
529 901 -0.7749710304193984 -0.16672013278193498
529 335 -0.8449725499930087 0.3699494099272971
529 944 -1.0232049330199793 0.07304594908940976
529 634 -0.8525210473437275 0.338918964962426
529 121 -0.9932392202000528 -1.4825096064661456
529 367 -0.7023562700693131 -0.8509306590977244
529 943 -0.7054131033784881 0.026262323358415607
529 613 -1.0232049330199793 -0.008825395939829672
529 853 -0.5528133380253024 -0.11108103503757416
529 42 -0.7449884249857615 -0.2325096064661456
529 857 -0.6318831518231445 -0.11408855383456684
529 159 -0.8254997971276126 -0.0645991587049517
529 47 -0.7004488287730841 -0.776627253524969
529 241 -0.7647955102604138 0.149069340902275

532 156 -0.15203660521039097 0.5395492170632665
532 137 -0.063668840497761 -0.062219751393681744
532 90 0.06260116668523796 -0.12044064094890405
532 37 -0.018538143921554173 0.4356190485046145
532 139 0.07687736671393175 0.3896708446616741
532 48 -0.059805670356619434 0.14415706020052088
532 60 0.04383881914448118 -0.38727151122805026
532 53 0.19719936685640915 -0.7825096064661454
532 92 -0.019707251278366922 0.512705704538639
532 97 0.1920277260163057 -0.09120525864005868
533 570 -0.5097025551594241 -0.14917627313281212
533 286 -0.3043548241728189 0.1424903935338544
533 64 -0.08977311997966231 0.2508237268671878
533 38 0.1192660008300317 0.5331153935338544
533 681 -0.03379445062052078 -0.9325096064661458
533 8 0.0564801692091117 0.19834145736364173
533 312 -0.06697548220301552 0.5519731521545439
533 614 -0.18894316632223163 0.08645591077523385
533 233 -0.5235447564177726 0.3239420064370804
533 31 -0.14050455747380441 -0.22609935005588921
533 224 0.21462669415057967 0.49788255039659957

536 890 -0.2660386262218557 0.2648588145864861
536 417 -0.20722654855382583 0.11749039353385449
536 824 -0.15092232421495932 0.6802810912082733
536 288 -0.39651612112412593 0.6769106833889267
536 1028 0.059330467880283067 0.4312834969821302
536 478 -0.30913305915855815 -0.3325096064661457
536 170 0.0028419327545328166 -0.16382828778482672
536 850 -0.15485534624974395 -0.042033415989954914
536 745 -0.21780209295563102 -0.3825096064661455
536 99 0.1483076112506354 0.8616764400454824
536 509 -0.2206121705687094 -0.2825096064661454
536 611 -0.29612372347930327 0.4122272356391177
536 700 0.006331576542807582 0.2049903935338544
536 237 -0.025518437953204298 0.21193483797829904
536 871 -0.19939583401105532 0.3174903935338542
536 554 -0.04192280940014337 -1.0088253959398297
536 1017 -0.07692700706830546 0.02860150464496547
536 455 0.16905040264339366 0.33144388190594753
536 694 0.16905040264339366 -0.16108103503757398
536 698 0.16905040264339366 0.3595956566917491
536 336 0.0012257841723477547

538 859 -0.47410676053876344 0.4312834969821302
538 205 -0.5043971811521679 -0.9682238921804314
538 892 -0.21025221637380254 0.261080137123598
538 59 -0.18422178985371973 -0.8528799768365158
538 738 -0.3012701850166901 0.061934837978298685
538 440 -0.1555088037510024 -1.5325096064661456
538 718 -0.6714012366394666 -0.2825096064661454
538 598 -0.8165459378795292 -0.3942743123484984
538 98 -0.5639248298025246 -0.3552368791934182
538 447 -0.24879970994107126 -0.5658429397994791
538 66 -0.8414151172703956 0.2187890948325557
538 150 -0.22004473292138815 -0.07074490058379279
538 776 -0.1920247106513071 0.2097980858415469
538 793 -0.8019093921038762 -0.07341869737523643
538 507 -0.6382569794020236 0.11008298612644696
538 376 -0.46726999571886285 -0.5182238921804312
538 402 -0.2535489142080225 -0.05393817789471722
538 767 -0.8165459378795292 -0.7444143683709075
538 142 -0.8414151172703956 0.017490393533854398
538 833 -0.6197364709373123 -0.21935171172930357
538 387 -0.5477835904874855 0.282196

In [176]:
better_MAE, better_RMSE = evaluation(better_result, test_user_dict, test_counter)    
print(better_MAE, better_RMSE)  # 0.6917506759527966 0.8983386235087089
# 0.7476839548925464 0.9730766472552652

0.6834455169311903 0.8918294879508567


Please explain what you do to improve the recommendation in 1-2 paragraphs.

The recommendation is improved mainly because I introduced extra information from movie info. When calculating the user bias. I came up with the idea that if current movie is similar with a movie that this user has rated before, that movie should have more weight when calculating the user bias. Since movie genres are specific terms, Jaccard Similarity is a good choice in this case. So I used Jaccard to measure the similarities between two movies. And if a movie has higher similarity, I will give it a higher weight in calculating the user bias. This method has a better MAE and is way more efficient than matrix factoriztion. 

## Collaboration declarations

*If you collaborated with anyone (see Collaboration policy at the top of this homework), you can put your collaboration declarations here.*

I found these websites useful when doing Part 2:  
https://datasciencemadesimpler.wordpress.com/tag/alternating-least-squares/  
http://www.quuxlabs.com/blog/2010/09/matrix-factorization-a-simple-tutorial-and-implementation-in-python/  
Collaborating with three classmates:  
Peixin Liu  
Yiran Huang  
Yicheng Li